In [64]:
from numpy.core.numeric import roll
from solid.objects import circle, cube, cylinder, linear_extrude, minkowski, offset, rotate, rotate_extrude, sphere, square, translate, import_scad, hull
from solid.utils import right, up, forward
from viewscad import Renderer


r = Renderer()
r.width = 600
r.height = 600

render = r.render
PI = 3.14


def spin(obj, amount=3):
    return sum((rotate((0, 0, a))(obj) for a in range(0, 360, 360//amount)))


def get_roll_offset(d,w,gap,roll):
    return (d-w-gap-roll)/2

def pump_shell(d, w, h,tube):
    inlet_offset = (d -  tube)/2 -w
    inlet = rotate((0,90,0))(cylinder(d=tube, h=d, segments=100))

    return cylinder(d=d, h=h, segments=100)\
        - cylinder(d=d-w*2, h=h, segments=100)\
        + cylinder(d=d, h=w, segments=100)\
        + cylinder(d=6, h=h/2, segments=100)\
        - translate((0,inlet_offset,h/2))(inlet)\
        - translate((0,-inlet_offset,h/2))(inlet)\

def pump_rolls(d, w, h, amount=3, roll=20, gap=1):
    offset = get_roll_offset(d,w,gap,roll)
    roll= cylinder(d=roll, h=h+w, segments=100)
    rolls = spin(right(offset)(roll), amount)

    return rolls

def pump_spinner(d, w, h, amount=3, roll=20, gap=1,key_size=6):
    offset = get_roll_offset(d,w,gap,roll)
    cutout = cylinder(d=roll+gap, h=h+w, segments=100)
    cutouts = spin(right(offset)(cutout), amount)

    return hull()(cutouts) - cutouts\
        + cylinder(d=d-w-gap, h=w, segments=100)\
        - up(w)(cylinder(d=key_size+gap,h=h,segments=100))\
        - cylinder(d=key_size,h=w,segments=6)



def peristaltic_pump(d=60, tube=12.5, tube_wall=2.5):
    height = PI*tube/2  # max size of squished tube
    padding = tube_wall*2
    shell = pump_shell(d=d, w=tube_wall, h=height+padding, tube=tube)
    spinner = pump_spinner(d=d-tube_wall*2, w=tube_wall, h=height)
    rolls = pump_rolls(d=d-tube_wall*2, w=tube_wall, h=height)

    return (shell, spinner, rolls)

(shell, spinner, rolls) = peristaltic_pump()

render(shell, outfile='./outfiles/peristaltic_pump_shell.stl')
render(spinner, outfile='./outfiles/peristaltic_pump_spinner.stl')
render(rolls, outfile='./outfiles/peristaltic_pump_rolls.stl')
